## RAP File Structure

In [1]:
import os
import urllib.request
from netCDF4 import Dataset

Let's download one file using the same query parameters than in the `rap` module. The bounding box is the one of the Western interconnect.

In [2]:
print("Beginning data download")
url = ("https://www.ncei.noaa.gov/thredds/ncss/model-rap130/202101/20210118/rap_130_20210118_0300_000.grb2?"
       "var=u-component_of_wind_height_above_ground&"
       "var=v-component_of_wind_height_above_ground&"
       "disableProjSubset=on&"
       "horizStride=1&"
       "addLatLon=true&"
       "accept=netCDF&"
       "north=49.8203&"
       "south=31.5543&"
       "west=-122.855&"
       "east=-101.353")
urllib.request.urlretrieve(url, "./rap_example.nc")

Beginning data download


('./rap_example.nc', <http.client.HTTPMessage at 0x11051d8e0>)

In [3]:
def nc_structure(data, verb=True):
    """Output data information.

    :param netCdF4.Dataset data: a netCDF4 dateset object.
    :param bool verb: whether information is printed.
    :return: (*tuple*) -- lists of data global attributes, dimensions and variables
    """

    def print_ncattr(key):
        """Prints data attributes for a given key.

        :param str key: unicode
        """
        try:
            print("\t\ttype:", repr(data.variables[key].dtype))
            for ncattr in data.variables[key].ncattrs():
                print("\t\t%s:" % ncattr, repr(data.variables[key].getncattr(ncattr)))
        except KeyError:
            print("\t\tWARNING: %s does not contain variable attributes" % key)

    # NetCDF global attributes
    nc_attrs = data.ncattrs()
    if verb:
        print("NetCDF Global Attributes:")
        for nc_attr in nc_attrs:
            print("\t%s:" % nc_attr, repr(data.getncattr(nc_attr)))
    nc_dims = [dim for dim in data.dimensions]  # list of nc dimensions
    # Dimension shape information.
    if verb:
        print("NetCDF dimension information:")
        for dim in nc_dims:
            print("\tName:", dim)
            print("\t\tsize:", len
                  (data.dimensions[dim]))
            print_ncattr(dim)
    # Variable information.
    nc_vars = [var for var in data.variables]  # list of nc variables
    if verb:
        print("NetCDF variable information:")
        for var in nc_vars:
            if var not in nc_dims:
                print("\tName:"), var
                print("\t\tdimensions:", data.variables[var].dimensions)
                print("\t\tsize:", data.variables[var].size)
                print_ncattr(var)
    return nc_attrs, nc_dims, nc_vars

In [4]:
data = Dataset("./rap_example.nc", "r")
nc_attrs, nc_dims, nc_vars = nc_structure(data)

NetCDF Global Attributes:
	Originating_or_generating_Center: 'US National Weather Service, National Centres for Environmental Prediction (NCEP)'
	Originating_or_generating_Subcenter: '0'
	GRIB_table_version: '2,1'
	Type_of_generating_process: 'Forecast'
	Analysis_or_forecast_generating_process_identifier_defined_by_originating_centre: 'Rapid Refresh (RAP)'
	Conventions: 'CF-1.6'
	history: 'Read using CDM IOSP GribCollection v3'
	featureType: 'GRID'
	History: 'Translated to CF-1.0 Conventions by Netcdf-Java CDM (CFGridWriter2)\nOriginal Dataset = /modeldata3/published/rapid-refresh/access/rap-130-13km/forecast/202101/20210118/rap_130_20210118_0300_000.grb2; Translation Date = 2021-03-01T18:05:30.662Z'
	geospatial_lat_min: 29.163452141623054
	geospatial_lat_max: 51.59138140554875
	geospatial_lon_min: -128.2667587035812
	geospatial_lon_max: -100.1816027596006
NetCDF dimension information:
	Name: time
		size: 1
		type: dtype('float64')
		units: 'Hour since 2021-01-18T03:00:00Z'
		standard_

In [5]:
nc_attrs

['Originating_or_generating_Center',
 'Originating_or_generating_Subcenter',
 'GRIB_table_version',
 'Type_of_generating_process',
 'Analysis_or_forecast_generating_process_identifier_defined_by_originating_centre',
 'Conventions',
 'history',
 'featureType',
 'History',
 'geospatial_lat_min',
 'geospatial_lat_max',
 'geospatial_lon_min',
 'geospatial_lon_max']

In [6]:
nc_dims

['time', 'height_above_ground5', 'y', 'x']

In [7]:
nc_vars

['u-component_of_wind_height_above_ground',
 'time',
 'height_above_ground5',
 'y',
 'x',
 'LambertConformal_Projection',
 'lat',
 'lon',
 'v-component_of_wind_height_above_ground']

In [8]:
data.close()
os.remove("./rap_example.nc")